In [30]:
import torch
import re
import tqdm

## Part 1: Data

We iterate through the Netflix dataset text files and prepare the matrix M, a matrix with columns of movies and users. If a user has watched a movie, the index corresponding to the (movie, user) pair will be set to 1, else 0.

### Data prep

We do a simple regex to get the movie patterns and then find all the users that have watched that movie.

In [31]:
movie_expression = re.compile(r"(\d+):")

def parse(lines: list):
    movie_id = None
    movies_and_users = []
    for line in tqdm.tqdm(lines):
        is_movie = movie_expression.search(line)
        if is_movie:
            movie_id = is_movie.groups()[0]
            continue
        user_id, _, _ = line.split(',')
        movies_and_users.append((int(movie_id), int(user_id)))
    return movies_and_users

In [32]:
files = [
    "dataset/combined_data_3.txt", # uncomment the below line to process the full dataset, but be warned it's huge.
#     "/kaggle/input/netflix-prize-data/combined_data_4.txt", "/kaggle/input/netflix-prize-data/combined_data_1.txt", "/kaggle/input/netflix-prize-data/combined_data_2.txt"
]
movies_and_users = []
for f in files:
    print(f'processing file {f}')
    with open(f, "r") as raw_text:
        lines = raw_text.readlines()
    movies_and_users.extend(parse(lines[:260578])) # even just one file is BIG so I've just been taking a subset of ~250,000 lines, remove the slice to process the full file
    print(f'completed processing file {f}')


processing file dataset/combined_data_3.txt


100%|██████████| 260578/260578 [00:00<00:00, 454040.59it/s]

completed processing file dataset/combined_data_3.txt


In [33]:
movies, users = zip(*movies_and_users)

In [34]:
print(movies_and_users[:10])
print(movies[:5])
print(users[:5])

[(9211, 1277134), (9211, 2435457), (9211, 2338545), (9211, 2218269), (9211, 441153), (9211, 1921624), (9211, 2096652), (9211, 818736), (9211, 284560), (9211, 1211224)]
(9211, 9211, 9211, 9211, 9211)
(1277134, 2435457, 2338545, 2218269, 441153)


We now have movie/user pairs. However, we need to turn the IDs into indexes. Lets do that below to convert this data into something we can index into a matrix.

In [35]:
unique_movies = sorted(list(set(movies)))
unique_users = sorted(list(set(users)))

movie_to_idx = {movie: i for i, movie in enumerate(unique_movies)}
user_to_idx = {user: i for i, user in enumerate(unique_users)}

normalised_movies = [movie_to_idx[m] for m in movies]
normalised_users = [user_to_idx[u] for u in users]

normalised_movies_and_users = list(zip(normalised_movies, normalised_users))

Now we need to create the matrix M representing the co-occurrence of a movie and a user.

In [36]:
M = torch.zeros(len(unique_movies), len(unique_users))
M.shape

torch.Size([55, 142475])

Lets populate the matrix.

In [37]:
for movie, user in normalised_movies_and_users:
    M[movie, user] = 1

# Recommendation

We have generated a matrix M of all films and all viewers, with the value of each index *i,j* corresponding to whether a given user *i* watched film *j*. This matrix is factorised into two smaller matrices U, F which represent embeddings for users and films respectively. We can iteratively hold each matrix constant while we tune the other to better match the results in M.

First, lets create our smaller matrices U and F.

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [39]:
embedding_dimension = 200

U = torch.randn(len(unique_users), embedding_dimension, requires_grad=True, device=device)
F = torch.randn(len(unique_movies), embedding_dimension, requires_grad=True, device=device)

In [40]:
print(F)

tensor([[-0.2763,  1.4851,  0.8592,  ...,  0.6936,  0.4123, -1.3256],
        [-1.3354, -0.4737,  1.7303,  ...,  0.0629,  0.4408,  1.1160],
        [ 0.1529, -0.1582,  0.4712,  ..., -1.2187,  1.0324,  0.2002],
        ...,
        [-0.7320, -0.3765, -0.2685,  ...,  0.7667, -1.1815, -0.9507],
        [-0.4715,  0.1284, -0.2211,  ..., -0.3990,  0.6457,  0.1315],
        [ 0.5461,  0.4456,  0.5588,  ..., -1.2041, -1.6152, -0.0072]],
       requires_grad=True)


We can see that our two matrices now exist to embed films and movies. Furthermore, when we transpose one matrix and multiply with the other, we get a matrix with shape equal to size M, see below.

In [41]:
M_hat = (F @ U.T)

We can use this to create a loss function between our created matrix M_hat and true matrix M, then backpropagate to the embedding networks.

We follow the Google tutorial [here](https://developers.google.com/machine-learning/recommendation/collaborative/matrix) and perform Weighted Matrix Factorization.

The 'weighted' element here comes from the loss function. Because our co-occurrences are distributed sparsely, our model has the option early to learn most effectively by simply classifying every (movie, user) pair as 0. So, we weight the loss from co-occurrences much more highly than the non-co-occurences.

The way we do this is by first masking out the non-occurrences and finding the difference between M and M_hat. This provides the loss for the co-occurrences. Next we do the inverse, masking out the co-occurrences to get the loss for the non-occurrences. Then we add the losses together in a weighted fashion. Simple, right!

In [42]:
M = M.to(device)
optim = torch.optim.Adam([U, F], 0.01)
# optim = torch.optim.AdamW([U, F], 0.01)

In [43]:
def training_loop(display):
    M_hat = F @ U.T
    difference = M - M_hat
    loss_1 = (difference * M) ** 2
    non_observation_mask = 1 - M
    loss_2 = (difference * non_observation_mask) ** 2
    loss = loss_1 + (0.005 * loss_2)
    loss = loss.mean()
    loss.backward()
    optim.step()
    optim.zero_grad()
    if display:
        print(loss.item())

In [44]:
for i in range(500):
    display = True if i % 100 == 0 else False
    training_loop(display)


7.837498664855957
0.16416901350021362
0.049234651029109955
0.017719188705086708
0.007146541960537434


The loss is decreasing, which means M_hat is increasingly similar to M. Lets check that out directly below.

In [45]:
M[:,0]

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.])

In [46]:
(F @ U.T)[:,0]

tensor([-0.6049,  0.5890, -0.0720,  0.6496,  0.2351, -0.2105,  0.9986,  0.1618,
         0.2783,  0.0288,  0.2330,  0.5394,  0.7391, -0.8873,  0.1142, -0.1725,
         0.5930, -0.6978, -0.6730,  0.0090,  0.1877, -0.3310,  0.1762, -0.6618,
         0.0756,  0.7980, -0.5159, -0.5512, -0.1191,  0.0893, -0.1093, -0.0622,
        -0.7080,  0.5581, -0.1991,  1.7017,  0.3186,  0.1285, -0.2221,  0.3530,
        -0.2360,  0.0185,  0.3755,  0.9985,  0.0926, -0.0700,  0.0249,  0.1023,
        -0.1634, -0.0845,  0.0854, -0.3388, -0.2578, -0.4634, -0.8617],
       grad_fn=<SelectBackward0>)

Sure enough, our positive co-occurrences are near 1, and our negative occurrences nearby 0. We did it! One additional potential improvement we could add is an element-wise sigmoid function to our M_hat to get all our outputs near 1.

In [47]:
sigmoid = torch.nn.Sigmoid()

def training_loop(display):
    M_hat = sigmoid(F @ U.T)
    difference = M - M_hat
    loss_1 = (difference * M) ** 2
    non_observation_mask = 1 - M
    loss_2 = (difference * non_observation_mask) ** 2
    loss = loss_1 + (0.005 * loss_2)
    loss = loss.mean()
    loss.backward()
    optim.step()
    optim.zero_grad()
    if display:
        print(loss.item())

In [48]:
for i in range(500):
    display = True if i % 100 == 0 else False
    training_loop(display)


0.0037430201191455126
0.0015117130242288113
0.001323428936302662
0.001205727458000183
0.0010979470098391175


In [49]:
sigmoid((F @ U.T)[:,0])

tensor([0.2467, 0.3234, 0.4167, 0.4811, 0.2838, 0.3243, 0.9743, 0.4059, 0.4315,
        0.4250, 0.3269, 0.7541, 0.4396, 0.2154, 0.2898, 0.5333, 0.2120, 0.1789,
        0.1518, 0.2833, 0.5068, 0.3030, 0.3453, 0.0461, 0.3297, 0.5003, 0.5712,
        0.3456, 0.3489, 0.8150, 0.4576, 0.2567, 0.1130, 0.6282, 0.2995, 0.7271,
        0.3502, 0.3140, 0.2128, 0.4433, 0.2292, 0.2017, 0.3230, 0.9657, 0.3362,
        0.2838, 0.3238, 0.6214, 0.2069, 0.7559, 0.2875, 0.1604, 0.2630, 0.1938,
        0.4128], grad_fn=<SigmoidBackward0>)

Interestingly, this doesn't actually provide us much of a benefit. In fact, it does a little worse than where we were before.

## Appendix

### Optimizer

The choice of optimizer happens to be very important. Try training again but substituting AdamW for the much more simple optimizer, SGD. You'll notice the training happens MUCH slower.

In [54]:
U = torch.randn(len(unique_users), embedding_dimension, requires_grad=True, device=device)
F = torch.randn(len(unique_movies), embedding_dimension, requires_grad=True, device=device)
optim = torch.optim.SGD([U, F], lr=0.01)

In [55]:
def training_loop(display):
    M_hat = F @ U.T
    difference = M - M_hat
    loss_1 = (difference * M) ** 2
    non_observation_mask = 1 - M
    loss_2 = (difference * non_observation_mask) ** 2
    loss = loss_1 + (0.005 * loss_2)
    loss = loss.mean()
    loss.backward()
    optim.step()
    optim.zero_grad()
    if display:
        print(loss.item())

In [52]:
for i in range(500):
    display = True if i % 100 == 0 else False
    training_loop(display)

7.5186028480529785
7.424254894256592
7.332400321960449
7.242955207824707
7.155838489532471


Interesting, right? Let's dig into this a bit more.

There are a few key differences between SGD and Adam. SGD on its own is effectively gradient descent at its most basic. It **only** takes the derivative of the loss with respect to each parameter and then descends the gradient to try and reach a minimum.

Adam is SGD with a few improvements:

1. Momentum: Adam essentially uses exponentially decaying gradients from previous updates to give the current weight update the context of previous updates. The consequence of this is that if previous updates tell us to adjust a weight in some way, and current updates tell us to adjust a weight in the same direction, the magnitude of the weight update will be increased. The key to this is the intuition of momentum -- previous motion in the weights has some impact on current motion.
2. Adaptive Learning Rates: If certain parameters are only seen rarely in learning, Adam will scale up the gradient update.

We can add momentum to our SGD algorithm. Lets see how much that gets us.

In [53]:
optim = torch.optim.SGD([U, F], momentum=0.9, lr=0.01)

for i in range(500):
    display = True if i % 100 == 0 else False
    training_loop(display)

7.070968151092529
6.383721828460693
5.7834391593933105
5.301899433135986
4.906630992889404


That's a large improvement. However, there's still a **big** difference between Adam and SGD. And this makes sense. Our matrix is largely sparse, and embeddings (especially for users) are updated only a few times each epoch. Having adaptive learning rates which learn a lot for those users makes sense. Lets take away the momentum and just look at adaptive learning. 

We can do this using AdaGrad, which solely modifies SGD to magnify rare (or regularly small) parameter updates.

In [57]:
optim = torch.optim.Adagrad([U, F], lr=0.01)

for i in range(500):
    display = True if i % 100 == 0 else False
    training_loop(display)

7.720456600189209
1.0821443796157837
0.6589827537536621
0.5082014799118042
0.4277366101741791


This gets us most of the way to Adam's performance! So, adaptive learning weights turn out to be *crucial* for learning in these sorts of factorization problems. Good to know!

Thank you for following along in this tutorial with me. I hope its been some help. Happy optimizing 👋🤖